# **Домашнее задание №1 (pro). Часть 1**

В этом домашнем задании (его первой части) вам будет необходимо:
*  обучить модель регрессии для предсказания стоимости автомобилей;


> Оценка за первую часть домашки = $min(\text{ваш балл}, 7)$



**Примечание**

В каждой части оцениваются как код, **так и ответы на вопросы.** Вопросы подсвечены синим цветом.

Если нет одного и/или другого, то часть баллов за соответствующее задание снимается.

In [5]:
!pip install ydata_profiling
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.9 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
from ydata_profiling import ProfileReport
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

random.seed(42)
np.random.seed(42)

### **Задание 0**
Давайте зафиксируем важный момент.

**Задание 0 (0 баллов).**
Изучите и ответье на вопрос: для чего фиксируем сиды в домашках?

`Your answer here` Сиды фиксируем для воспроизводимости результатов

# **Часть 1 | EDA**


Первая часть состоит из классических шагов EDA:

- Базовый EDA и обработка признаков (1.7 балла)
- Визуализации признаков и их анализ (1.6 балл)

Всего можно набрать 3.3 основных балла и 0.25 бонусных.

В следующих частях, вы увидите бонусные задания. Бонусные задания выделены как **Дополнительное задание/Бонус**. Вы можете выполнять их, чтобы в случае ошибок в основных задачах всё равно набрать за работу максимум. Кроме того, дополнительные задания позволяют вам углубить знания.

Призываем активно использовать их!

## **Простейший EDA и обработка признаков (1.7 балла)**

In [7]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_test.csv')

print("Train data shape:", df_train.shape)
print("Test data shape: ", df_test.shape)

Train data shape: (6999, 13)
Test data shape:  (1000, 13)


### **Задание 1.(0.4 балла)**
Выполните операции, направленные на базовое исследование данных:

- [ ] Посмотрите, есть ли в датасете пропуски и дубликаты. Запишите/выведите названия колонок, для которых есть пропущенные значения (0.1 балла)
- [ ] Посмотрите, есть ли в данных явные дубликаты (0.05 балла)
- [ ] Постройте дашборд в одну строку, используя [ydata-profilling](https://github.com/ydataai/ydata-profiling)(0.15 балла)
- [ ] Опишите базовые выводы (какие — выберите сами), используя дашборд (0.2 балла).

**Ваш ответ здесь.**

In [8]:
miss = df_train.isna().sum()
print(miss[miss > 0])

dupl = df_train.duplicated().sum()
print(f"\nдубли обычные: {dupl}")

cols = [
    'year',
    'km_driven',
    'fuel',
    'transmission',
    'owner',
    'max_power',
    'engine',
    'name'
]

un_dupl = df_train[df_train.duplicated(subset=cols, keep=False)]

print(f"\nеявные дубли: {un_dupl.shape[0]}")
print(un_dupl.sort_values(by=cols).head())

mileage      202
engine       202
max_power    196
torque       203
seats        202
dtype: int64

дубли обычные: 985

еявные дубли: 1807
                           name  year  selling_price  km_driven    fuel  \
4574  Mitsubishi Lancer 2.0 GLd  1999         150000     198000  Diesel   
6436  Mitsubishi Lancer 2.0 GLd  1999         150000     198000  Diesel   
6856             Maruti Alto LX  2000          45957      90000  Petrol   
6875             Maruti Alto LX  2000          45957      90000  Petrol   
899      Toyota Qualis Fleet A3  2000         200000     100000  Diesel   

     seller_type transmission         owner    mileage   engine max_power  \
4574  Individual       Manual  Second Owner  14.8 kmpl  1998 CC    68 bhp   
6436  Individual       Manual  Second Owner  14.8 kmpl  1998 CC    68 bhp   
6856  Individual       Manual  Second Owner  19.7 kmpl   796 CC  46.3 bhp   
6875  Individual       Manual  Second Owner  19.7 kmpl   796 CC  46.3 bhp   
899   Individual       Man

In [9]:
profile = ProfileReport(df_train, title="masheenky")
profile.to_file("cars.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 13/13 [00:00<00:00, 20.15it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Достаточно много пропущенных, количество дубликатов также приличное.  есть некоторое количество корреляци, но они логичные исходя из темы датасета. некоторые столбцы странно форматированы и их нужно отредактировать  например крутящий момент мжно либо удалить обороты двс либо разделить на два столбца


### **Задание 2 (0.2 балла)**
Проанализируйте статистики датасета.

**Ваша задача:**
- [ ] Посчитайте основные статистики по числовым столбцам для трейна и теста
- [ ] Посчитайте основные статистики по категориальным столбцам для трейна и теста
- [ ] Сравните среднее и медиану внутри `train`, внутри `test` и между собой. О чём могут говорить результаты?
- [ ] Сделайте выводы по статистикам. Отсутсвие выводов равносильно по баллам невыполнению задания.

**Подсказка:**

Используте ``.describe()`` с нужным(и) аргументом(-ами).

In [10]:
# Ваш код здесь
print("\nтреин числовые")
print(df_train.describe(include=['number']).round(2))

print("\nтест числовые")
print(df_test.describe(include=['number']).round(2))

print("\nтреин категори")
print(df_train.describe(include=['object']).round(2))

print("\nтест категори")
print(df_test.describe(include=['object']).round(2))


треин числовые
          year  selling_price   km_driven    seats
count  6999.00        6999.00     6999.00  6797.00
mean   2013.82      639515.20    69584.62     5.42
std       4.05      808941.91    57724.00     0.97
min    1983.00       29999.00        1.00     2.00
25%    2011.00      254999.00    35000.00     5.00
50%    2015.00      450000.00    60000.00     5.00
75%    2017.00      675000.00    97000.00     5.00
max    2020.00    10000000.00  2360457.00    14.00

тест числовые
          year  selling_price  km_driven   seats
count  1000.00        1000.00    1000.00  981.00
mean   2013.68      617901.04   71393.34    5.41
std       4.01      758553.86   48486.22    0.92
min    1995.00       31000.00    1303.00    4.00
25%    2011.00      250000.00   37000.00    5.00
50%    2014.00      434999.00   61500.00    5.00
75%    2017.00      670000.00  100000.00    5.00
max    2020.00     6000000.00  375000.00    9.00

треин категори
                          name    fuel seller_type tr

In [11]:
'''
 сравнения внутри:
 треин среднее по цене выше чем медиана значит есть некоторые очень дорогие продажи\выбросы
 треин средний год не сильно оличается от медианы
 треин средний  пробег также выше медины как и с ценой значит есть машины с большим количеством пробега
 треин среднее количество сидень не сильно отличается от медианы

 тестовая выборка показывает такие же выводы что и треин и между ними нет особо большой разницы в значениях значит будущая модель в контексте данных будет хорошо предсказывать


категории показывают:

mileage, engine, max_power и torque являются текстовыми (о чем я выше писал) и имеют пропуски
также авто на дизеле с механикой доминируют в выборке

 '''

'\n сравнения внутри: \n треин среднее по цене выше чем медиана значит есть некоторые очень дорогие продажи\\выбросы\n треин средний год не сильно оличается от медианы\n треин средний  пробег также выше медины как и с ценой значит есть машины с большим количеством пробега\n треин среднее количество сидень не сильно отличается от медианы\n\n тестовая выборка показывает такие же выводы что и треин и между ними нет особо большой разницы в значениях значит будущая модель в контексте данных будет хорошо предсказывать\n\n\nкатегории показывают:\n\nmileage, engine, max_power и torque являются текстовыми (о чем я выше писал) и имеют пропуски\nтакже авто на дизеле с механикой доминируют в выборке \n\n '

### **Задание 3 (0.2 балла)**

- [ ] Посмотрите, есть ли в трейне объекты с одинаковым признаковым описанием (целевую переменную следует исключить). Если есть, то сколько? (0.01 балла)
- [ ] Отобразите такие объекты (0.01 балла)
- [ ] Удалите повторяющиеся строки. Если при одинаковом признаковом описании цены на автомобили отличаются, то оставьте первую строку по этому автомобилю (0.01 балла)
- [ ]  Обновите индексы строк таким образом, чтобы они шли от 0 без пропусков (0.01 балла)
- [ ] Подумайте, могут ли в данных быть другие скрытые дубли? Предложите, как их можно отлавливать. (0.16 балла)

P.S тут данные без подвоха, но выводы нам нужны.

In [12]:
f_cols = df_train.columns.drop('selling_price')

mask_dupl = df_train.duplicated(subset=f_cols, keep=False)

df_dupl = df_train[mask_dupl]

n_dupl = df_dupl.shape[0]

n_u_dupl = df_dupl.drop_duplicates(subset=f_cols).shape[0]

print(f"все повторяющиеся строки: {n_dupl}")
print(f"уникальные: {n_u_dupl}")

print("\nпримеры дублирующихся объектов")
print(df_dupl.sort_values(by=list(f_cols)).head(5))

df_train_clean = df_train.drop_duplicates(subset=f_cols, keep='first')

removed = df_train.shape[0] - df_train_clean.shape[0]

print(f"новый размер {df_train_clean.shape[0]} строк")

df_train_clean = df_train_clean.reset_index(drop=True)

'''
могут быть различные скрытые дубли за счет вариаций написания крутящего момента моделей авто и тд
думаю для поиска таких вещей уже есть библиотеки готовые которые учитывают регистр повтор символов и тд
а с другие которые можно вручную отредактировать надо отредактирова
'''

все повторяющиеся строки: 1799
уникальные: 640

примеры дублирующихся объектов
                                      name  year  selling_price  km_driven  \
1703  Audi Q3 2.0 TDI Quattro Premium Plus  2017        2825000      22000   
6307  Audi Q3 2.0 TDI Quattro Premium Plus  2017        2825000      22000   
1836               Audi Q5 3.0 TDI Quattro  2014        1850000      76131   
6704               Audi Q5 3.0 TDI Quattro  2014        1850000      76131   
117             Audi Q5 35TDI Premium Plus  2018        3975000      31800   

        fuel seller_type transmission        owner     mileage   engine  \
1703  Diesel      Dealer    Automatic  First Owner  15.73 kmpl  1968 CC   
6307  Diesel      Dealer    Automatic  First Owner  15.73 kmpl  1968 CC   
1836  Diesel  Individual    Automatic  First Owner  13.22 kmpl  2967 CC   
6704  Diesel  Individual    Automatic  First Owner  13.22 kmpl  2967 CC   
117   Diesel      Dealer    Automatic  First Owner  17.01 kmpl  1968 CC   

 

'\nмогут быть различные скрытые дубли за счет вариаций написания крутящего момента моделей авто и тд \nдумаю для поиска таких вещей уже есть библиотеки готовые которые учитывают регистр повтор символов и тд\nа с другие которые можно вручную отредактировать надо отредактирова\n'

In [13]:
#df_train = df_train.drop_duplicates(subset=X_train.columns, keep='first')

In [14]:
#assert df_train.shape == (5840, 13)

### **Задание 4 (0.3 балла)**

Вы могли заметить, что с признаками ``mileage, engine, max_power и torque`` всё не очень хорошо. Они распознаются как строки (можно убедиться в этом, вызвав `data.dtypes`). Однако эти переменные не являются категориальными — они — числа. Соответственно, нужно привести их к числовому виду.

**Задача :**
* [ ] Уберите единицы измерения для признаков ``mileage, engine, max_power``.
* [ ] Приведите тип данных к ``float``.
* [ ] Предобработайте признак `torque` — разделите его на два: собственно `torque` и `max_torque_rpm`. Учтите единицы измерения


**Важно**
- Все действия нужно производить над обоими датасетами — `train` и `test`.

In [15]:
# your code here

dataframes = [df_train_clean, df_test]

for df in dataframes:

    df['mileage'] = df['mileage'].astype(str).str.replace(r' kmpl| km/kg', '', regex=True)
    df['engine'] = df['engine'].astype(str).str.replace(r' CC', '', regex=False)
    df['max_power'] = df['max_power'].astype(str).str.replace(r' bhp', '', regex=False)

    df['mileage'] = pd.to_numeric(df['mileage'], errors='coerce')
    df['engine'] = pd.to_numeric(df['engine'], errors='coerce')
    df['max_power'] = pd.to_numeric(df['max_power'], errors='coerce')

    df['torque_clean'] = df['torque'].astype(str).str.lower()
    df['torque_clean'] = df['torque_clean'].str.replace(' at ', ' ', regex=False).str.replace('@', ' ', regex=False)
    df['torque_clean'] = df['torque_clean'].str.replace(r'[\(\),]', '', regex=True).str.strip()

    df['torque'] = df['torque_clean'].str.extract(r'^(\d+\.?\d*)', expand=False)
    df['torque'] = pd.to_numeric(df['torque'], errors='coerce')

    df['max_torque_rpm'] = df['torque_clean'].str.extract(r'(\d+)-?\d*\s*rpm', expand=False)

    df['max_torque_rpm'] = pd.to_numeric(df['max_torque_rpm'], errors='coerce')

    df.drop('torque_clean', axis=1, inplace=True) # это помощник помог

print(df_train_clean[['mileage', 'engine', 'max_power', 'torque', 'max_torque_rpm']].head())
print(df_train_clean[['mileage', 'engine', 'max_power', 'torque', 'max_torque_rpm']].dtypes)

print(df_test[['mileage', 'engine', 'max_power', 'torque', 'max_torque_rpm']].head())
print(df_test[['mileage', 'engine', 'max_power', 'torque', 'max_torque_rpm']].dtypes)

   mileage  engine  max_power  torque  max_torque_rpm
0    23.40  1248.0      74.00  190.00          2000.0
1    21.14  1498.0     103.52  250.00          1500.0
2    23.00  1396.0      90.00   22.40          1750.0
3    16.10  1298.0      88.20   11.50             NaN
4    20.14  1197.0      81.86  113.75          4000.0
mileage           float64
engine            float64
max_power         float64
torque            float64
max_torque_rpm    float64
dtype: object
   mileage  engine  max_power  torque  max_torque_rpm
0     14.0  2498.0      112.0   260.0          1800.0
1     21.5  1497.0      108.5   260.0          1500.0
2     12.9  1799.0      130.0   172.0          4300.0
3     25.1  1498.0       98.6   200.0          1750.0
4     16.5  1172.0       65.0    96.0          3000.0
mileage           float64
engine            float64
max_power         float64
torque            float64
max_torque_rpm    float64
dtype: object


### **Задание 5 (0.3 балла)**

На первом шаге мы обнаружили пропуски. Давайте избавимся от них.

**Задание:**
- [ ] Заполните пропуски в столбцах медианами. Убедитесь, что после заполнения пропусков не осталось.
- [ ] Почему стоит применять именно медиану. Могли ли мы применить среднее? Обоснуйте свое рассуждение.
- [ ] Как правильно считать медиану для заполнения? Выберите верное утверждение:
 - По тестовым свою, по тренировочным — свою - ПРАВИЛЬНО
 - По тренировочным данным для `train` и `test`






In [16]:
# your code here

df_train1 = df_train_clean.copy()
df_test1 = df_test.copy()

cols2 = ['seats', 'mileage', 'engine', 'max_power', 'torque', 'max_torque_rpm']

med_train = df_train1[cols2].median()
med_test = df_test1[cols2].median()

df_train1[cols2] = df_train1[cols2].fillna(med_train)
df_test1[cols2] = df_test1[cols2].fillna(med_test)

train_na = df_train1[cols2].isna().sum().sum()
print(f"пропусков в df_train1 {train_na}")

test_na = df_test1[cols2].isna().sum().sum()
print(f"пропусков в df_test1 {test_na}")

пропусков в df_train1 0
пропусков в df_test1 0


### **Задание 6 (0.2 балла)**

Теперь, когда не осталось пропусков, давайте преобразуем столбцы к более подходящим типам. А именно столбцы ``engnine`` и ``seats`` к приведем к `int`.

- [ ] Осуществите приведение столбцов к необходимому типу.
- [ ] Ответье на вопрос — почему (хоть мы этого и не делаем) ``seats``, возможно рассмотреть как категориальную переменную?

In [17]:
# your code here

df_train1['engine'] = df_train1['engine'].astype(int)
df_train1['seats'] = df_train1['seats'].astype(int)

df_test1['engine'] = df_test1['engine'].astype(int)
df_test1['seats'] = df_test1['seats'].astype(int)

'''
мне кажется что количество посадочных мест в авто возможно рассматривать как категорию, потому что количество мест в авто влияет на сам тип авто. будет это
спорт кар или семейный микроавтобус от чего цена зависит тоже

'''

'\nмне кажется что количество посадочных мест в авто возможно рассматривать как категорию, потому что количество мест в авто влияет на сам тип авто. будет это \nспорт кар или семейный микроавтобус от чего цена зависит тоже\n\n'

`Your answer here`

### **Задание 7 (0.1 балла)**

Снова вызовите метод describe и проанализируйте статистики.

**Ответье на вопрос:**
- [ ] Есть ли основания предполагать, что заполнение пропусков свдинуло наши распределения? Могло ли это вообще возникнуть?

In [18]:
print(df_train1.describe(include=['number']).round(2))

print(df_test1.describe(include=['number']).round(2))

'''
да оно сдвинуло но минимально потому что до этого оно было без особых хвостов
почти любое заполнение пропусков ведет к тому что распределние сдвигается
'''

          year  selling_price   km_driven  mileage   engine  max_power  \
count  5840.00        5840.00     5840.00  5840.00  5840.00    5840.00   
mean   2013.43      522960.09    73952.24    19.43  1429.45      87.91   
std       4.10      535432.00    60071.14     3.99   485.66      31.64   
min    1983.00       29999.00        1.00     0.00   624.00       0.00   
25%    2011.00      250000.00    39000.00    16.95  1197.00      68.00   
50%    2014.00      405000.00    70000.00    19.37  1248.00      81.86   
75%    2017.00      640000.00   100000.00    22.30  1498.00      99.00   
max    2020.00    10000000.00  2360457.00    42.00  3604.00     400.00   

        torque    seats  max_torque_rpm  
count  5840.00  5840.00         5840.00  
mean    161.31     5.43         2624.85  
std      91.17     0.98         1078.13  
min       4.80     2.00          175.00  
25%      99.04     5.00         1750.00  
50%     150.00     5.00         2000.00  
75%     200.00     5.00         3500.00

'\nда оно сдвинуло но минимально потому что до этого оно было без особых хвостов\nпочти любое заполнение пропусков ведет к тому что распределние сдвигается\n'

`Your answer here`

## **Визуализации и корреляция (1.6 балла + 0.25)**

Визуализация данных — важный шаг в работе. Визуализировать данные необходимо, например, чтобы:

- Оценить распределения признаков самих по себе (это может натоклнуть вас на мысли о модели, которую можно использовать)
- Сравнить распределения на `train` и `test` — чтобы проверить, насколько информация, на которой вы будете обучаться согласуется с той, на которой модель должна работать
- Оценить есть ли явная связь признаков с целевой переменной

**Важно:**

Если распределения на `train` и `test` не совпадают, это не значит, что нужно перемешивать данные! Более корректно актуализировать задачу и уточнить, а не устарели ли данные `train`. Также полезным может быть собрать новую тестовую выборку, смешав те, что имеются сейчас.

**Если вы будете подгонять распределения, то можете встретиться с переобучением!**

### **Задание 8 (0.5 балла)**

Шаг 1.
- [ ] Воспользуйтесь `pairplot` из библиотеки `seabron`, чтобы визуализировать попарные распределения числовых признаков для `train`
- [ ] По полученному графику ответьте на вопросы:
 - Можно ли предположить на основе распределений связь признаков с целевой переменной?
 - Можно ли предположить на основе распределений выдвинуть гипотезу о корреляциях признаков?

Шаг 2.

- [ ] Постройте pairplot по тестовым данным
- [ ] Ответьте на вопрос "Похожими ли оказались совокупности при разделении на трейн и тест?"

In [ ]:
# your code here

df_train_plot = df_train1.select_dtypes(include=np.number).copy()

sns.pairplot(df_train_plot)
plt.suptitle('train', y=1, fontsize=15)
plt.show()

'''
можно сказать что мощность двигателя коррелирует от крутящаего момента потому что это зависимые переменные
чем больше объем двигателя тем скорее будет выше крутящий момент и также лс
чем больше км пройдено тем машина старше и дешевле
чем новее авто тем оно дороже продается
да похожи
'''

In [ ]:
df_test_plot = df_test1.select_dtypes(include=np.number).copy()

sns.pairplot(df_test_plot)
plt.suptitle('test', y=1, fontsize=15)
plt.show()

### **Задание 9 (0.5 балла)**

И так, вы выдвинули гипотезы о наличии связи. Теперь давайте оценим эту связь в числах.

**Задание:**
- [ ] Получите значения коэффициента корреляции Пирсона для тренировочного набора данных при помощи `pd.corr()`
- [ ] По полученным корреляциям постройте тепловую карту (`heatmap` из бибилотеки seaborn)

In [54]:
# your code here

numeric_features = df_train1.select_dtypes(include=np.number).columns

correlation_matrix = df_train1[numeric_features].corr()
print(correlation_matrix.round(2))

                year  selling_price  km_driven  mileage  engine  max_power  \
year            1.00           0.43      -0.37     0.34    0.00       0.16   
selling_price   0.43           1.00      -0.16    -0.10    0.45       0.69   
km_driven      -0.37          -0.16       1.00    -0.18    0.23       0.02   
mileage         0.34          -0.10      -0.18     1.00   -0.57      -0.37   
engine          0.00           0.45       0.23    -0.57    1.00       0.68   
max_power       0.16           0.69       0.02    -0.37    0.68       1.00   
torque          0.25           0.61       0.04    -0.13    0.61       0.74   
seats           0.04           0.15       0.19    -0.45    0.65       0.24   
max_torque_rpm  0.06          -0.22      -0.25     0.01   -0.47      -0.25   

                torque  seats  max_torque_rpm  
year              0.25   0.04            0.06  
selling_price     0.61   0.15           -0.22  
km_driven         0.04   0.19           -0.25  
mileage          -0.13  -0.

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    linewidths=.5,
    cbar_kws={'label': 'коэффициент корреляции Пирсона'} # тут чат использовал чтобы красоту сразу навёл
)
plt.title('train heatmap', fontsize=15)
plt.show()

- [ ] Ответьте на вопросы:
 - Какие 2 признака наименее скоррелированы между собой?
 - Между какими наблюдается довольно сильная положительная линейная зависимость?
 - Правильно ли, опираясь на данные, утверждать, что чем меньше год, тем, скорее всего, больше километров проехала машина к дате продажи?
 - Изучите типы корреляций в `pd.corr()`. Какую вы использовали по умолчанию?

менее скоррелированы признаки engine и year (0.00), mileage и max_torque_rpm (0.01)

сильная положительная наблюдается между max_power и torque (0.74), max_power и selling_price (0.69), а также max_power и engine (0.68)

да верно

пирсона

### **Задание 10 (0.6 балла)**

По умолчанию `pd.corr` возвращает корреляцию Пирсона, говорящую о линейной взаимосвязи. Но зависимости существуют не только линейные! В этой задаче, попробуйте измерить другие способы вычисления корреляций.

- [ ] Реализуйте корреляцию Спирмена/Кендала (на выбор) без использования библиотек (можно пользоваться только `numpy`). Сравните результаты вычисления с библиотечной реализацией
- [ ] Сделайте выводы


Постройте матрицу корреляции [phik](https://pypi.org/project/phik/)
- [ ] Проинтерпретируйте результаты


In [22]:
import phik
from phik import phik_matrix

In [ ]:
# your code here

df_phik = df_train1.copy()

interval_cols = df_phik.select_dtypes(include=np.number).columns.tolist()

phik_mat = phik.phik_matrix(df_phik, interval_cols=interval_cols)

plt.figure(figsize=(12, 10))
sns.heatmap(
    phik_mat,
    annot=True,
    cmap='viridis',
    fmt=".2f",
    linewidths=0.5
)
plt.title('phik heatmap', fontsize=18)
plt.show()

лучшими предикторами цены показались:max_power (0.85) engine (0.78) transmission (0.65) torque (0.65) year (0.58), fuel (0.58) owner (0.50)
значит их скорее надо оставить для будущей модели


сильная корреляция engine и max_power (0.88) seats и engine (0.65) fuel и transmission (0.65)


### **Дополнительные визуализации (бонус 0.25 балла)**

Если вам кажется, что мы не попросили вас нарисовать какие-то очень важные зависимости, нарисуйте их и поясните.
Один график: 0.125 балла, при условии, что он обоснован.

In [24]:
# your code here

# **Часть 2 (1.7 балла) | Модель только на вещественных признаках**

В этой части вам предстоит обучить модель только на вещественных признаках. Почему только на них?

Чем больше признаковое пространство — чем сложнее модель. А чем модель проще — тем лучше для скорости работы и интерпретации признаков.

За задания этой части вы можете набрать 1.7 основных и 0.15 бонусных балла;

### **Задание 11 (0.05 балла)**

Сделайте на тренировочный и тестовый наборы. Она уже даны, достаточно просто отделить целевой признак. Перед разбиением создайте копию датафрейма, который будет хранить только вещественные признаки и используйте его (то есть категориальные столбцы (все, кроме seats) необходимо удалить).

В переменные y_train и y_test запишите значения целевых переменных. Используйте уже имеющиеся данные train и test. Если считаете уместным применить какую-то более хитрую технику разбиения — примените, но опишите, зачем.

In [25]:
features = ['year', 'km_driven', 'max_power', 'seats']

X_train = df_train1[features].copy()
y_train = df_train1['selling_price']


In [26]:
# опишите выбранные вещественные признаки
'''
year: является важным фактором амортизации чем новее год тем выше цена в данном файле

km_driven: измеряет износ автомобиля предиктор который имеет отрицательную связь с ценой

max_power: сильный предиктор цены

seats: разделяет стандартные автомобили (5 мест) и минивэны/внедорожники (7 мест)

выбрал такие потому что мультиколлинеарность тут не должна проявится
'''

'\nyear: является важным фактором амортизации чем новее год тем выше цена в данном файле\n\nkm_driven: измеряет износ автомобиля предиктор который имеет отрицательную связь с ценой\n\nmax_power: сильный предиктор цены\n\nseats: разделяет стандартные автомобили (5 мест) и минивэны/внедорожники (7 мест)\n\nвыбрал такие потому что мультиколлинеарность тут не должна проявится\n'

In [27]:
y_test = df_test1['selling_price']
X_test = df_test1[features].copy()

### **Задание 12. (0.2 балла)**

Построим нашу первую модель!
- [ ] Обучите классическую линейную регрессию с дефолтными параметрами. Посчтитайте $R^2$ и $MSE$ для трейна и для теста.
- [ ] Сделайте выводы по значениям метрик качества.

**Примечание:**

Здесь и далее $R^2$ и $MSE$ для трейна и для теста выводите везде, где требуется обучать модели, даже если в явном виде этого не просят. Иначе непонятно, как понять, насколько успешны наши эксперименты.

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error as MSE

# your code here

model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

r2_train = r2_score(y_train, y_train_pred)
mse_train = MSE(y_train, y_train_pred)

r2_test = r2_score(y_test, y_test_pred)
mse_test = MSE(y_test, y_test_pred)

print(f"R2 test: {r2_test:.4f}")
print(f"MSE test: {mse_test:.4f}")

print(f"R2: {r2_train:.4f}")
print(f"MSE: {mse_train:.4f}")



R2 test: 0.5914
MSE test: 234886096923.1267
R2: 0.5883
MSE: 118000864436.3056


### **Задание 13 (0.15 балла)**

- [ ] Реализуйте $R^2$ руками. Приведите формулу $R^2$ и объясните каждую компоненту метрики

`Your answer here`

### **Бонус (0.15 балла)**

- [ ] Реализуйте [$\text{adjusted}-R^2$](https://en.wikipedia.org/wiki/Coefficient_of_determination).
- [ ] Объясните, когда применяется $\text{adjusted}-R^2$?

In [29]:
# your code here

`Your answer here`

### **Задание 14 (0.05 балла)**

Всегда есть место совершенству. Поэтому давайте попробуем улучшить модель. При помощи стандартизации признаков.

- [ ] Стандартизируйте значения в тренировочных и тестовых данных. Стандартизатор **обучайте только на `train`**.

In [30]:
from sklearn.preprocessing import StandardScaler

# your code here

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

r2_train = r2_score(y_train, y_train_pred)
mse_train = MSE(y_train, y_train_pred)

r2_test = r2_score(y_test, y_test_pred)
mse_test = MSE(y_test, y_test_pred)

print(f"R2 test: {r2_test:.4f}")
print(f"MSE test: {mse_test:.4f}")

print(f"R2: {r2_train:.4f}")
print(f"MSE: {mse_train:.4f}")

R2 test: 0.5914
MSE test: 234886096923.1256
R2: 0.5883
MSE: 118000864436.3057


### **Задание 15 (0.1 балла)**

Хотя стандартизация не помогла сильно прибавить в качестве она открыла возможность интерпретировать важность признаков в модели. Правило интерпретации такое:

Чем больше коэффициент $\beta_i$ по модулю, тем важнее признак.

**Ответьте на вопрос:**

- [ ] Какой признак оказался наиболее информативным в предсказании цены?

In [31]:
# your code here

coefs = model.coef_
importance = pd.DataFrame({
    'feature': features,
    'coef': coefs,
})

print(importance.sort_values('coef', ascending=False))

'''
мощность двигателя оказалась наиболее информативный
'''

     feature           coef
2  max_power  347599.786336
0       year  160890.306923
3      seats   -4149.966509
1  km_driven  -36632.374298


'\nмощность двигателя оказалась наиболее информативный\n'

### **Задание 16 (0.25 балла)**

Попробуем улучшить нашу модель с помощью применения регуляризации. Для этого воспльзуемся `Lasso` регрессией.  Кроме того, попробуйте использовать её теоретическое свойство отбора признаков, за счет зануления незначимых коэффициентов.

**Задание:**

- [ ] Обучите Lasso регрессию на тренировочном наборе данных с нормализованными признаками. Оцените её качество
- [ ] Проверьте, занулила ли L1-регуляризация с параметрами по умолчанию какие-нибудь веса? Предположите почему.

In [32]:
from sklearn.linear_model import Lasso

# your code here

lasso = Lasso()
lasso.fit(X_train_scaled, y_train)

y_train_pred_lasso = lasso.predict(X_train_scaled)
y_test_pred_lasso = lasso.predict(X_test_scaled)

r2_train_lasso = r2_score(y_train, y_train_pred_lasso)
mse_train_lasso = MSE(y_train, y_train_pred_lasso)

r2_test_lasso = r2_score(y_test, y_test_pred_lasso)
mse_test_lasso = MSE(y_test, y_test_pred_lasso)

print("lasso R2 train:", r2_train_lasso)
print("lasso MSE train:", mse_train_lasso)
print("lasso R2 test:", r2_test_lasso)
print("lasso MSE test:", mse_test_lasso)

lasso R2 train: 0.5883283962787671
lasso MSE train: 118000864439.78056
lasso R2 test: 0.5913795220297297
lasso MSE test: 234886719155.94128


In [33]:
coef_df = pd.DataFrame({
    'feature': features,
    'coef': lasso.coef_
})
print(coef_df)

     feature           coef
0       year  160889.785649
1  km_driven  -36631.751693
2  max_power  347598.580456
3      seats   -4148.770419


### **Задание 17. Финальный рывок (0.4 балла)**

До этого мы с вами использовали `train` для обучения и `test` для прогнозирования. Но у нас есть ещё одна задача — подобрать оптимальные параметры модели. Для этого используем кросс-валидацию, описанную на семинарах.

Кроме того, выжмем максимум из модификаций регрессии. Построим `ElasticNet`. И сделаем всё по порядку.

**Ваша задача 1:**

- [ ] Перебором по сетке (c 10-ю фолдами) подберите оптимальные параметры для Lasso-регрессии. Вам пригодится класс [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
- [ ] Ответьте на вопросы:
 - Сколько грид-сёрчу пришлось обучать моделей?
 - Что значит каждый параметр `param_grid`, который вы заполняли для `GridSearch`? Опишите каждый
 - Какой коэффициент регуляризации у лучшей из перебранных моделей? Занулились ли какие-нибудь из весов при такой регуляризации?

In [34]:
# your code here
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10, 50] # сила L1
}

lasso = Lasso(max_iter=10000)

grid = GridSearchCV(
    estimator=lasso,
    param_grid=param_grid,
    cv=10,
    scoring='neg_mean_squared_error'
)

grid.fit(X_train_scaled, y_train)

best_lasso = grid.best_estimator_

print("лучший alpha:", grid.best_params_['alpha'])

coef_df = pd.DataFrame({
    'feature': features,
    'coef': best_lasso.coef_,
})

print(coef_df)


'''

количество моделей = alpha * cv = 120 обученных моделей

alpha коэффициент L1-регуляризации
чем больше alpha тем сильнее штрафуются коэффициенты
'''

лучший alpha: 50
     feature           coef
0       year  160861.148789
1  km_driven  -36602.821590
2  max_power  347539.457151
3      seats   -4089.722565


'\n\nколичество моделей = alpha * cv = 120 обученных моделей\n\nalpha коэффициент L1-регуляризации\nчем больше alpha тем сильнее штрафуются коэффициенты\n'

In [35]:
param_grid = {
    "alpha": [0.01, 0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10, 20],  # сила L1
    "max_iter": [1000, 5000, 10000, 20000],                # макс число итераций оптимизатора
}

lasso = Lasso()

grid = GridSearchCV(
    estimator=lasso,
    param_grid=param_grid,
    cv=10,
    scoring="neg_mean_squared_error"
)

grid.fit(X_train_scaled, y_train)
best_lasso = grid.best_estimator_

print("лучшие параметры gridcv:", grid.best_params_)

coef_df = pd.DataFrame({
    'feature': features,
    'coef': best_lasso.coef_,
})

print(coef_df)

'''
количество моделей = параметры(160) * cv = 1600 обученных моделей
'''

лучшие параметры gridcv: {'alpha': 20, 'max_iter': 1000}
     feature           coef
0       year  160880.838844
1  km_driven  -36619.435460
2  max_power  347575.676725
3      seats   -4126.179686


'\nколичество моделей = параметры(160) * cv = 1600 обученных моделей\n'

**Ваша задача 2:**

- [ ] Перебором по сетке (c 10-ю фолдами) подберите оптимальные параметры для [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) регрессии.
- [ ] Ответьте на вопрос:
 - Сколько грид-сёрчу пришлось обучать моделей?
 - Какие гиперпараметры соответствуют лучшей (по выбранной метрике качества) из перебранных моделей?

In [36]:
# your code here
from sklearn.linear_model import ElasticNet

param_grid = {
    "alpha": [0.01, 0.05, 0.1, 0.2, 0.5, 1, 5, 10, 20],       # сила L1
    "max_iter": [1000, 5000, 10000],              # число итераций
}

elastic = ElasticNet()

grid = GridSearchCV(
    estimator=elastic,
    param_grid=param_grid,
    cv=10,
    scoring="neg_mean_squared_error"
)

grid.fit(X_train_scaled, y_train)

best_enet = grid.best_estimator_

print("kучшие параметры elasticmet:", grid.best_params_)

coef_df = pd.DataFrame({
    'feature': features,
    'coef': best_lasso.coef_,
})

print(coef_df)

'''
количество моделей = параметры(160) * cv = 840 обученных моделей
'''

kучшие параметры elasticmet: {'alpha': 0.01, 'max_iter': 1000}
     feature           coef
0       year  160880.838844
1  km_driven  -36619.435460
2  max_power  347575.676725
3      seats   -4126.179686


'\nколичество моделей = параметры(160) * cv = 840 обученных моделей\n'

### **Задание 18 (0.35 балла)**

И так, вы обучили все варианты регуляризаций, доступные в sklearn, но не все, что мы обсудили на занятии! И следующее задание о забытой - $L0$ регуляризации.


**Ваша задача:**>
- [ ] Реализуйте модель с $L0$-регуляризаицей.
- [ ] Обучите модель на данных и проанализируйте результат, экспериментируя с коэффициенитом регуляризации
- [ ] Проанализируйте результаты обучения

**Важно:**

Вопреки тому, что L0 не реализована в стандартных пакетах, концепция данной регуляризации не является мертвой. Она может встречаться в [статьях](https://arxiv.org/abs/1712.01312) и экспериментах.



In [37]:
# your code here



# **Часть 3 (0.5 балла) | Добавляем категориальные фичи**

Попробуем для улучшения модели дать ей больше признаков. Добавим категориальные фичи.

За эту часть можно набрать 0.5 основных балла.



### **Задание 19 (0.1 балла)**

Проанализируйте столбец `name`. Очевидно, что эта переменная является категориальной, однако категорий в ней много.

- [ ] Предобработайте столбец `name`, чтобы избежать его удаления

In [38]:
# your code here
df_train1['brand'] = df_train1['name'].str.split().str[0].str.lower().str.strip()
df_test1['brand'] = df_test1['name'].str.split().str[0].str.lower().str.strip()

In [39]:
#assert X_train_cat.shape == (5840, 11)

In [40]:
#X_train_cat.describe(include='object')

### **Задание 20 (0.1 балла)**

- [ ] Закодируйте категориалльные фичи и ``seats`` методом OneHot-кодирования.

In [41]:
from sklearn.preprocessing import OneHotEncoder # или можно использовать get_dummies из библиотеки pandas

# your code here

num_cols = ['year', 'km_driven', 'max_power']
cat_cols = ['fuel', 'transmission', 'owner', 'seller_type', 'seats', 'brand']

all_cols = num_cols + cat_cols

X_train_full = df_train1[all_cols].copy()
X_test_full = df_test1[all_cols].copy()

X_train_full['seats'] = X_train_full['seats'].astype(str)
X_test_full['seats']  = X_test_full['seats'].astype(str)

encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

X_train_cat = encoder.fit_transform(X_train_full[cat_cols])
X_test_cat  = encoder.transform(X_test_full[cat_cols])

feat_out = encoder.get_feature_names_out(cat_cols)

X_train_cat_df = pd.DataFrame(X_train_cat, columns=feat_out, index=X_train_full.index)
X_test_cat_df  = pd.DataFrame(X_test_cat, columns=feat_out, index=X_test_full.index)

X_train_enc = pd.concat([X_train_full[num_cols], X_train_cat_df], axis=1)
X_test_enc  = pd.concat([X_test_full[num_cols], X_test_cat_df], axis=1)

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


### **Задание 21 (0.2 балла)**

OHE — базовый алгоритм преобразования категориальных признаков, но и с ним нужно быть аккуратными.

**Ответьте на вопросы:**


* Как корректно работать с OHE преобразованием?
* Почему мы удаляем один столбец?
* Пусть из $n$ признаков мы получили $n-1$ столбец, из которых $k < n -1$ оказались не важными по весам модели. Корректно ли их удалить?

`Your answer here`

### **Задание 22 (0.1 балла)**
Повторим то, что делали на прошлом шаге для моделей на вещественных признаках, однако теперь с моделью `Ridge`.


**Ваша задача:**
- [ ] Переберите параметр регуляризации `alpha` для гребневой (ridge) регрессии с помощью класса `GridSearchCV` В качестве параметров при объявлении GridSearchCV кроме модели укажите метрику качества $R^2$. Кроссвалидируйтесь по 10-ти фолдам.
- [ ] Ответье на вопрос: Удалось ли улучшить качество прогнозов?

In [42]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# your code here

param_grid = {
    'alpha': [0.01, 0.1, 1, 5, 10, 50, 100, 200, 500, 1000]
}

ridge = Ridge()

grid = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid,
    scoring='r2',
    cv=10,
    n_jobs=-1
)

grid.fit(X_train_enc, y_train)

best_ridge = grid.best_estimator_

print("лучшее alpha:", grid.best_params_['alpha'])
print("лучший средний R2 на кроссвалидации:", grid.best_score_)

y_train_pred = best_ridge.predict(X_train_enc)
y_test_pred  = best_ridge.predict(X_test_enc)

r2_train = r2_score(y_train, y_train_pred)
mse_train = MSE(y_train, y_train_pred)

r2_test = r2_score(y_test, y_test_pred)
mse_test = MSE(y_test, y_test_pred)

print("R2 train:", r2_train)
print("MSE train:", mse_train)

print("R2:", r2_test)
print("MSE:", mse_test)

'''
все серьезно улучшилось
но я использовал тут и категориальные признаки чтобы посмотреть уже пряи наа всё
'''

лучшее alpha: 1
лучший средний R2 на кроссвалидации: 0.7115078660662473
R2 train: 0.7701907082831443
MSE train: 65872153516.92235
R2: 0.7807888677660451
MSE: 126008818521.91237


'\nвсе серьезно улучшилось\nно я использовал тут и категориальные признаки чтобы посмотреть уже пряи наа всё\n'

# **Часть 4 - бонусная (1 балл) | Feature Engineering**

В этой части домашнего задания вам предлагается проявить свою креативность для улучшения прогноза модели. Любые другие модели, кроме различных форм линейной (или полиномиальной) регресси, использовать запрещается. А значит, придется работать с признаками

**Что можно попробовать сделать?** (каждый пункт по 0.4 балла, но не больше 1-х балла в сумме)

1.   *Сгенерировать новые признаки на основе уже существующих:*
    * посчитать произведения // частные признаков (кажется, что посчитать число "лошадей" на литр объема может быть полезно);
    * имеет смысл обратить внимание на визуализации в части с EDA (к примеру, зависимость цены от года выглядит квадратичной, а не линейной; значит, квадрат года нам, скорее всего, принесет больше пользы)

2.   *Добыть новые признаки:*
    * имеем название автомобиля, которое никак не используем (можно спарсить инфу о классе автомобиля или каких-то специфических опциях)
    * можно добавить пороговые признаки вроде "владелец третий или больше" и объединить признаки в некоторые осмысленные правила, например "первый или второй владелец и продавец официальный дилер" (подбирать пороги удобно по диаграммам рассеяния)

3.   *Поработать с уже имеющимися:*
    * далеко не факт, что заполнить пропуск медианой было лучшей идеей (как минимум, можно добавить dummy-столбец для модели, сигнализирующий, что раньше на месте медианы был пропуск -- там где он был, конечно); попробуйте другие способы филлинга;
    * мы не анализировали, есть ли в данных выбросы => никак выбросы не обрабатывали; наиболее простым и, тем не менее, довольно полезным вариантом нахождения выбросов могут послужить boxplot'ы для каждого столбца; что делать с выбросами думайте сами :) -- вариантов довольно много
    * мы толком не смотрели на таргет сам по себе; в нем тоже могут быть неожиданности -- стоит хотя бы проверить
    * можно заметить, что некоторые признаки распределены совсем не нормально; возможно их стоит отлогарифмировать

И так далее...

Feel Free to Try!

In [43]:
# your code here

# **Часть 4. | Бизнесовая (0.5 балла)**

### **Задание 23 (0.25 балла)**

В мире бизнеса очень важно давать оценку качества модели понятную бизнесу, поэтому иногда заказчики приходят с кастомными метриками. Попробуем сделать такую для нашей задачи.

**Описание метрики:**

Среди всех предсказанных цен на авто нужно посчитать долю прогнозов, отличающихся от реальных цен на эти авто не более чем на 10% (в одну или другую сторону)

**Ваша задача:**

- [ ] Реализуйте метрику `business_metric`
- [ ] Посчитайте метрику для всех обученных моделей и определеите, какаю лучше всего решает задачу бизнеса

In [44]:
'''
def business_metrics(y_true, y_pred):
    # your code here

business_metrics(# your code here)
'''

'\ndef business_metrics(y_true, y_pred):\n    # your code here\n\nbusiness_metrics(# your code here)\n'

### **Задание 24 (0.25 балла)**

Но у бизнеса не всегда есть идеи и иногда задача на выбор метрики делегируется вам.

**Задание:**

- [ ] Придумайте и реализуйте другую кастомную метрику с учетом того, что модель не должна сильно ошибаться в прогнозе, но недопрогноз для модели, согласно мнению бизнеса, хуже, чем перепрогноз.
- [ ] Посчитайте метрику для всех обученных моделей и определеите, какаю лучше всего решает задачу бизнеса



In [45]:
'''
def your_business_metrics(y_true, y_pred):

business_metrics(# your code here)
'''

'\ndef your_business_metrics(y_true, y_pred):\n\nbusiness_metrics(# your code here)\n'

# **<font color="green">Часть 5 | Создание интерактивного приложения на Streamlit (3 балла)</font>**

Вам необходимо создать интерактивное приложение на Streamlit, которое будет:

- Показывать основные информативные графики/гистограммы в рамках EDA (1 балл)
- На вход запрашивать csv-файл с признаками объектов или запрашивать признаки объекта в окошках для ввода, и применять на поступивших объектах модель (1 балл)
- Визуализировать веса обученной модели (1 балл)

# **Часть 6 | Оформление репозитория и оценка сервиса (3 балла)**

**Результаты вашей работы** необходимо разместить в своём GitHub-репозитории. В этот же репозиторий позже будет добавлена вторая часть.

Под результатами первой части понимаем следующее:

---

### Обязательные файлы:

1. **`.ipynb`-ноутбук** со всеми экспериментами:

   * полный EDA,
   * все шаги препроцессинга,
   * обучение и сравнение моделей,
   * сохранённые output’ы.

2. **`.pickle`-файл**, содержащий:

   * обученную модель (или пайплайн `scaler + model`);
   * параметры скейлинга;
   * любые числовые объекты, необходимые для инференса внутри Streamlit-приложения.

3. **`.md`-файл с выводами** о проделанной работе:

   * что было сделано (краткое описание каждого этапа);
   * какие результаты были получены (метрики + интерпретация);
   * что дало наибольший прирост качества;
   * что сделать не удалось и почему (это нормально и даже полезно);
   * **оценка разработанного сервиса**:

     * насколько приложение удобно в использовании;
     * что получилось визуализировать хорошо, а что — менее удачно;
     * какие ограничения или проблемы вы заметили;
     * какие улучшения планируете в следующей итерации.

По результатам второй части, ждем:

4. **Streamlit-приложение** (например, `app.py`), которое можно запустить командой:

   ```bash
   streamlit run app.py
   ```

В приложении должно быть:

* отображение ключевых графиков EDA;
* ввод данных (CSV или ручной ввод);
* применение модели;
* визуализация весов/коэффициентов.

**И ссылку на приложение в StreamlitHub.**


### **За что могут быть сняты баллы:**

* отсутствие `.pickle` с моделью / пайплайном;
* слабая или неполная аналитика в `.md`-файле;
* беспорядок в ноутбуке или неясная логика вычислений;
* отсутствие возможности запустить Streamlit-приложение.


ТУТ Я ПРОДОЛЖУ СДЕЛАВ ТО ЧТО НЕ СДЕЛАЛ РАНЕЕ ЧТОБЫ УСПЕТЬ ПОД ДЕДЛАЙН(на баллы не надеюсь) и чуть-чуть попытаюсь, что нибудь улучшить

In [46]:
num_features = ['year', 'km_driven', 'mileage', 'engine', 'max_power', 'torque', 'max_torque_rpm']
cat_features = ['fuel', 'seller_type', 'transmission', 'owner', 'brand', 'seats']
features = num_features + cat_features

X_train_cb = df_train1[features].copy()
y_train_cb = df_train1['selling_price']

X_test_cb = df_test1[features].copy()
y_test_cb = df_test1['selling_price']

for col in cat_features:
    X_train_cb[col] = X_train_cb[col].astype(str)
    X_test_cb[col] = X_test_cb[col].astype(str)

In [47]:
model_base = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='RMSE',
    eval_metric='R2',
    cat_features=cat_features,
    random_seed=42,
    verbose=100
)

model_base.fit(
    X_train_cb, y_train_cb,
    eval_set=(X_test_cb, y_test_cb),
    early_stopping_rounds=50,
    use_best_model=True
)

y_pred_base = model_base.predict(X_test_cb)

r2_base = r2_score(y_test_cb, y_pred_base)
mse_base = mean_squared_error(y_test_cb, y_pred_base)
mape_base = np.mean(np.abs((y_test_cb - y_pred_base) / y_test_cb)) * 100

print("\n" + "=" * 40)
print(f"CatBoost R2: {r2_base:.4f}")
print(f"CatBoost MAPE: {mape_base:.2f}%")
print("=" * 40)

0:	learn: 0.1289805	test: 0.1051863	best: 0.1051863 (0)	total: 59.9ms	remaining: 59.9s
100:	learn: 0.9447489	test: 0.9564453	best: 0.9564546 (99)	total: 1.46s	remaining: 13s
200:	learn: 0.9624435	test: 0.9654279	best: 0.9654553 (198)	total: 3.77s	remaining: 15s
300:	learn: 0.9687820	test: 0.9688962	best: 0.9688962 (300)	total: 5.08s	remaining: 11.8s
400:	learn: 0.9725617	test: 0.9706786	best: 0.9706786 (400)	total: 6.21s	remaining: 9.27s
500:	learn: 0.9755272	test: 0.9715103	best: 0.9715332 (497)	total: 7.33s	remaining: 7.3s
600:	learn: 0.9777574	test: 0.9721942	best: 0.9722171 (591)	total: 8.47s	remaining: 5.63s
700:	learn: 0.9792415	test: 0.9725215	best: 0.9725531 (697)	total: 9.6s	remaining: 4.09s
800:	learn: 0.9803301	test: 0.9727662	best: 0.9727886 (796)	total: 10.7s	remaining: 2.67s
900:	learn: 0.9814649	test: 0.9732893	best: 0.9732893 (900)	total: 11.9s	remaining: 1.31s
999:	learn: 0.9823931	test: 0.9736879	best: 0.9737284 (990)	total: 13s	remaining: 0us

bestTest = 0.9737283906

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

sns.scatterplot(x=y_test_cb, y=y_pred_base, alpha=0.5, ax=axes[0], color='#3498db')
min_val = min(y_test_cb.min(), y_pred_base.min())
max_val = max(y_test_cb.max(), y_pred_base.max())
axes[0].plot([min_val, max_val], [min_val, max_val], color='#e74c3c', lw=2, linestyle='--', label='идеальное предсказание')
axes[0].set_title(f'Base Model: R2 = {r2_base:.3f}', fontsize=14)
axes[0].set_xlabel('реальная цена')
axes[0].set_ylabel('предсказанная цена')
axes[0].legend()
axes[0].grid(alpha=0.3)

residuals = y_test_cb - y_pred_base
sns.histplot(residuals, kde=True, ax=axes[1], color='#2ecc71', bins=30)
axes[1].axvline(0, color='#e74c3c', linestyle='--', lw=2, label='0 ошибка')
axes[1].set_title('распределение ошибок', fontsize=14)
axes[1].set_xlabel('ошибка')

plt.tight_layout()
plt.show()

In [49]:
def create_features(df):
    df_new = df.copy()
    df_new['age'] = 2020 - df_new['year']
    df_new['power_per_liter'] = df_new['max_power'] / (df_new['engine'] / 1000 + 1e-3)
    df_new['km_per_year'] = df_new['km_driven'] / (df_new['age'] + 1)
    return df_new

df_train_new = create_features(df_train1)
df_test_new = create_features(df_test1)

num_features_new = ['year', 'km_driven', 'mileage', 'engine', 'max_power', 'torque', 'max_torque_rpm',
                    'age', 'power_per_liter', 'km_per_year']
features_new = num_features_new + cat_features

X_train_opt = df_train_new[features_new].copy()
y_train_opt = df_train_new['selling_price']

X_test_opt = df_test_new[features_new].copy()
y_test_opt = df_test_new['selling_price']

for col in cat_features:
    X_train_opt[col] = X_train_opt[col].astype(str)
    X_test_opt[col] = X_test_opt[col].astype(str)

In [50]:
model_opt = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=5,
    subsample=0.8,
    colsample_bylevel=0.8,
    loss_function='RMSE',
    eval_metric='R2',
    cat_features=cat_features,
    random_seed=42,
    verbose=200
)

model_opt.fit(
    X_train_opt,
    np.log1p(y_train_opt),
    eval_set=(X_test_opt, np.log1p(y_test_opt)),
    early_stopping_rounds=100,
    use_best_model=True
)

y_pred_log = model_opt.predict(X_test_opt)
y_pred_opt = np.expm1(y_pred_log)

0:	learn: 0.0384558	test: 0.0276214	best: 0.0276214 (0)	total: 11.1ms	remaining: 22.2s
200:	learn: 0.9102469	test: 0.9285114	best: 0.9285114 (200)	total: 3.84s	remaining: 34.4s
400:	learn: 0.9228242	test: 0.9371437	best: 0.9371437 (400)	total: 5.86s	remaining: 23.4s
600:	learn: 0.9293669	test: 0.9406223	best: 0.9406223 (600)	total: 7.95s	remaining: 18.5s
800:	learn: 0.9343284	test: 0.9429472	best: 0.9429501 (798)	total: 10.1s	remaining: 15.2s
1000:	learn: 0.9384494	test: 0.9442484	best: 0.9442484 (1000)	total: 12.2s	remaining: 12.2s
1200:	learn: 0.9415205	test: 0.9448539	best: 0.9448622 (1191)	total: 15.2s	remaining: 10.1s
1400:	learn: 0.9443403	test: 0.9456232	best: 0.9456234 (1399)	total: 18.2s	remaining: 7.8s
1600:	learn: 0.9465088	test: 0.9460474	best: 0.9460474 (1600)	total: 20.4s	remaining: 5.08s
1800:	learn: 0.9484133	test: 0.9463099	best: 0.9463099 (1800)	total: 22.6s	remaining: 2.5s
1999:	learn: 0.9503306	test: 0.9466299	best: 0.9466365 (1990)	total: 24.7s	remaining: 0us

best

In [53]:
r2_opt = r2_score(y_test_opt, y_pred_opt)
mse_opt = mean_squared_error(y_test_opt, y_pred_opt)
mape_opt = np.mean(np.abs((y_test_opt - y_pred_opt) / y_test_opt)) * 100

def business_metric(y_true, y_pred, threshold=0.10):
    absolute_percentage_error = np.abs((y_true - y_pred) / y_true)
    correct_predictions = np.sum(absolute_percentage_error <= threshold)
    return correct_predictions / len(y_true) * 100

bus_score_base = business_metric(y_test_cb, y_pred_base)
bus_score_opt = business_metric(y_test_opt, y_pred_opt)

results_df = pd.DataFrame({
    'Metric': ['R2 Score', 'MSE', 'MAPE (%)', 'Biz Metric (<10% err)'],
    'Ridge (Linear)': [0.7807, 126008818521, '---', '---'],
    'CatBoost (Base)': [r2_base, mse_base, mape_base, bus_score_base],
    'CatBoost (Optimized)': [r2_opt, mse_opt, mape_opt, bus_score_opt]
})

print(results_df.round(4))

feature_importance = pd.DataFrame({
    'feature': features_new,
    'importance': model_opt.get_feature_importance()
}).sort_values('importance', ascending=False)

                  Metric Ridge (Linear)  CatBoost (Base)  CatBoost (Optimized)
0               R2 Score         0.7807     9.737000e-01          9.576000e-01
1                    MSE   126008818521     1.510167e+10          2.438930e+10
2               MAPE (%)            ---     1.524820e+01          1.491030e+01
3  Biz Metric (<10% err)            ---     4.870000e+01          4.590000e+01


In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x='importance', y='feature', data=feature_importance.head(10), palette='viridis')
plt.title('топ важных признаков (Optimized Model)')
plt.show()

In [52]:
import pickle

cols_to_fill = ['seats', 'mileage', 'engine', 'max_power', 'torque', 'max_torque_rpm']
medians = df_train1[cols_to_fill].median().to_dict()

df_sample = df_train1.sample(1000, random_state=42)

data_to_save = {
    'model': model_opt,
    'medians': medians,
    'df_sample': df_sample,
    'cat_features': cat_features,
    'num_features': num_features_new
}

with open('car_price_model.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)